In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
import math
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import datasets
from datetime import date, datetime
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt
plt.style.use("seaborn")
sns.set(style="white")

## Perfil de peliculas y usuarios.

### Perfil de las peliculas segun su genero 

In [2]:
df_genres = pd.read_csv("./data/movie_genres.csv")
df_movies = pd.read_csv("./data/movies.csv")
df_ratings = pd.read_csv("./data/ratings_train.csv")

In [3]:
# Creo una matriz donde se agrego un flag "1" en cada genero del que la pelicula pertenece.
# Perfil de la pelicula sobre el Genero
for g in df_genres["genre"].unique():
    mids = df_genres.loc[df_genres["genre"] == g, "movieID"].unique()
    df_movies["Genre {}".format(g)] = df_movies["id"].isin(mids).astype("int16")
df_movies.head()
# Podria ponderar por la cantidad de generos que tiene, si tiene 4 generos cada uno seria 0.25

,id,title,year,imdbID,rtID,Genre Adventure,Genre Animation,Genre Children,Genre Comedy,Genre Fantasy,...,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
0,1,Toy story,1995,114709,toy_story,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,113497,1068044-jumanji,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpy Old Men,1993,107050,grumpy_old_men,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale,1995,114885,waiting_to_exhale,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,1995,113041,father_of_the_bride_part_ii,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Perfil de los usuarios

In [9]:
df_ratings.head()


,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,1,51198,608,5.0,17,9,1997,23,40,20
1,2,51198,1073,4.0,17,9,1997,23,41,6
2,3,51198,141,4.0,17,9,1997,23,41,58
3,4,51198,356,4.0,17,9,1997,23,43,44
4,5,51198,1198,5.0,17,9,1997,23,44,35


In [64]:
# Voy a crea el perfil de un usuario
USUARIO = 75
df_ratings_usuario = df_ratings[df_ratings["userID"] == USUARIO]
df_ratings_usuario.sort_values("rating").head()

,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
473065,525416,75,6225,0.5,29,10,2006,23,21,25
473058,525408,75,3,1.0,29,10,2006,23,17,16
473060,525410,75,2054,1.5,29,10,2006,23,17,33
473081,525435,75,3258,1.5,29,10,2006,23,26,8
473054,525403,75,160,2.0,29,10,2006,23,16,52


In [67]:
# Ahora lo que busco es condensar sus gustos de pelicula por los generos.
# Mergeo al rating de un usuario el perfil de cada pelicula
df_ratings_usuario = df_ratings[df_ratings["userID"] == USUARIO].merge(df_movies, left_on="movieID", right_on="id")
df_ratings_usuario.sort_values("rating").head()

,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second,...,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
13,525416,75,6225,0.5,29,10,2006,23,21,25,...,0,0,0,0,0,0,0,0,0,0
6,525408,75,3,1.0,29,10,2006,23,17,16,...,0,0,0,0,0,0,0,0,0,0
8,525410,75,2054,1.5,29,10,2006,23,17,33,...,0,0,1,0,0,0,0,0,0,0
27,525435,75,3258,1.5,29,10,2006,23,26,8,...,0,0,0,0,0,0,0,0,0,0
2,525403,75,160,2.0,29,10,2006,23,16,52,...,0,1,1,0,0,0,0,0,0,0


### Ahora lo que busco es agrupar toda esta tabla en una sola lista, donde pueda sumar los generos para tener para un solo usuario el perfil completo.

In [18]:
# Veo para el usuario 75 que porcentaje tiene sobre los generos Action, Comedy y Drama de las peliculas que opino.
df_ratings_usuario["Genre Action"].mean(), df_ratings_usuario["Genre Comedy"].mean(), df_ratings_usuario["Genre Drama"].mean()

(0.7333333333333333, 0.3111111111111111, 0.3333333333333333)

In [74]:
# Una mejor manera de ranquear, es tomar el minimo y el maximo raking que el usuario puso y no el 1-5 que es el estandar. Si un usuario siempre vota de 3-5, entonces se usa esa base.

(df_ratings_usuario["Genre Action"] * df_ratings_usuario["rating"]).mean()


2.6666666666666665

In [83]:
# Si saco la media, me da CERO en los que no tiene accion y un numero de votacion sobre los que si tenian accion. Idealmente no deberia contar los CEROs.
# Lo Normalizo por la cantidad de votaciones posibles
df_ratings_usuario.loc[df_ratings_usuario["Genre Action"]==1, "rating"].mean() / 5.0, df_ratings_usuario.loc[df_ratings_usuario["Genre Drama"]==1, "rating"].mean() / 5.0, df_ratings_usuario.loc[df_ratings_usuario["Genre Comedy"]==1, "rating"].mean() / 5.0

(0.7272727272727273, 0.7533333333333333, 0.6357142857142857)

### Esto va generado varios vectores entre CERO y UNO y el vector del usuario. Luego deberia buscar cual es el vector mas cercano.

In [85]:
# Perfil del USER 75, la idea es generar el perfil para los usuarios.
# Creo PERFIL como diccionario
PERFIL ={}
for c in df_ratings_usuario.columns:
    # Me quedo solo con las columnas que empiezan con GENRE 
    if not c.startswith("Genre"):
       continue

    PERFIL[c] = [df_ratings_usuario.loc[df_ratings_usuario[c]==1, "rating"].mean() / 5.0]

print(PERFIL)



{'Genre Action': [0.7272727272727273], 'Genre Adventure': [0.6571428571428571], 'Genre Animation': [0.9], 'Genre Children': [0.3], 'Genre Comedy': [0.6357142857142857], 'Genre Fantasy': [0.625], 'Genre Romance': [0.5], 'Genre Drama': [0.7533333333333333], 'Genre Crime': [0.8300000000000001], 'Genre Thriller': [0.7761904761904762], 'Genre Horror': [0.7], 'Genre Mystery': [0.55], 'Genre Sci-Fi': [0.6538461538461539], 'Genre IMAX': [nan], 'Genre Documentary': [nan], 'Genre War': [0.7333333333333333], 'Genre Musical': [0.9], 'Genre Film-Noir': [nan], 'Genre Western': [0.5], 'Genre Short': [nan]}


In [86]:
# Lo convierto en un DataFrame
# fillna(0) me llena los nan como CERO
df_perfil = pd.DataFrame.from_dict(PERFIL).fillna(0)
df_perfil

,Genre Action,Genre Adventure,Genre Animation,Genre Children,Genre Comedy,Genre Fantasy,Genre Romance,Genre Drama,Genre Crime,Genre Thriller,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
0,0.727273,0.657143,0.9,0.3,0.635714,0.625,0.5,0.753333,0.83,0.77619,0.7,0.55,0.653846,0.0,0.0,0.733333,0.9,0.0,0.5,0.0


### Este es el perfil del usuario 75, e informa segun lo que califico cuales son los generos que estan mas cerca de su gusto.
### Ahora lo que busco es cuales son las peliculas que estan mas cerca de este usuario usando la Similitud Coseno (1 es super similar)

In [91]:
# Agarro una pelicula cualquier y me fijo que tan cerca o lejos esta de su perfil.
MOVIE = 65088
movie = df_movies.query(f"id=={MOVIE}")
columnas_genero = [c for c in df_movies.columns if c.startswith("Genre")]
A = movie[columnas_genero].values[0]
B = df_perfil[columnas_genero].values[0]
# A es la pelicula y B es el perfil del usuario

In [92]:
# El perfil
# Similaridad Coseno, mientras mas cerca del 1 mejor
np.dot(A,B) / (np.linalg.norm(A) * np.linalg.norm(B))

0.33397865808365

In [ ]:
# Ahora busco hacerlo para todas las peliculas

columnas_genero = [c for c in df_movies.columns if c.startswith("Genre")]
B = df_perfil[columnas_genero].values[0]

for index, row in df_movies.iterrows():    
    movie = df_movies.query("id == {}".format(row["id"]))

    A = movie[columnas_genero].values[0]
    #print(row["id"], np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B)))

#0.49472772819267147 * 5.0
# Si tomo el mayor y lo multiplico por 5, eso me da el rating.

# Tambien podria tomar las 5 con mayor rating y podria decir que esas son las 5 que mas le deberian gustar.
# No deberia mostrarle peliculas que YA vio, porque esas podrian ser muy cercanas.
# Esta tecnica me permite con 1 sola pelicula, ya empezar a recomendarle peliculas similares.

In [ ]:
# Con el pais podria hacer algo similar y decir para cada usuario que pais de pelicula prefiere. Pero quizas no haya que hacer * 5.0?
# Podria pondear los generos por 0.6 y los paises por 0.4 ...
# Como reflejar en el perfil del usuario que prefiere peliculas a partir del año 2000?
# Se podrian hacer bines por decadas. Agregar el año en el perfil, etc. 
# (año- minimoAño / maximoAño - minimoAños) los paso a CERO y UNO, y mutiplico año contra año.
# Podria incluir
# Año, directores, actores, paises, etc.

# Calculo para todos los usuarios